# Reinforcement Learning on Google Cloud

Ran from datalab. You can do this from cloud using `datalab create ...` Or here are steps to get datalab running locally:

1. `sudo docker run -it -p "127.0.0.1:8081:8080" -v "${HOME}:/content"      -e "PROJECT_ID=[YOUR_PROJECT_ID]"  gcr.io/cloud-datalab/datalab:local`
2. Click user image at top right of datalab, and login to you GCP account. Gives you authentification for everything :)

In [1]:
%%bash
# Install packages to test model locally.
apt-get update
apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig libffi-dev
pip install gym
pip install gym[atari]

pip install opencv-python
apt update && apt install -y libsm6 libxext6
apt-get install -y libxrender-dev
pip install keras

Reading package lists...
  Using cached https://files.pythonhosted.org/packages/a7/bf/eadbc04a632b2e6818efc731c101e652b1135e9d0a23f8fd32042034a313/opencv_python-3.4.4.19-cp27-cp27mu-manylinux1_x86_64.whl
Reading package lists...
  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3f/c4/2ff40221029f7098d58f8d7fb99b97e8100f3293f9856f0fb5834bef100b/Keras_Applications-1.0.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/94/74e0fa783d3fc07e41715973435dd051ca89c550881b3454233c39c73e69/Keras_Preprocessing-1.0.5-py2.py3-none-any.whl


E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Exception:
Traceback (most recent call last):
  File "/usr/local/google/home/praneetdutta/.local/lib/python2.7/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/usr/local/google/ho

### First run locally to make sure everything is working.

In [ ]:
%%bash
rm -r rl_on_gcp/my_model
gcloud ml-engine local train \
   --module-name=trainer.trainer \
   --package-path=${PWD}/rl_on_gcp/trainer \
   --\
   --steps=100000\
   --start_train=100\
   --buffer_size=100\
   --save_model=True\
   --model_dir='my_model'

### Run on ML-Engine

In [5]:
%%bash
JOBNAME=rl_breakout_$(date -u +%y%m%d_%H%M%S)
REGION='us-central1'
BUCKET='dqn-breakout'

gcloud ml-engine jobs submit training $JOBNAME \
        --package-path=$PWD/rl_on_gcp/trainer \
        --module-name=trainer.trainer \
        --region=$REGION \
        --staging-bucket=gs://$BUCKET \
        --scale-tier=BASIC_TPU\
        --runtime-version=1.9 \
        --\
        --steps=5000000\
        --start_train=10000\
        --buffer_size=1000000\
        --save_model=True\
        --model_dir='gs://dqn-breakout/models/'

jobId: rl_breakout_181207_061313
state: QUEUED


Job [rl_breakout_181207_061313] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe rl_breakout_181207_061313

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs rl_breakout_181207_061313


#### TODO: why doesn't it use GPU?
From logs using `BASIC_GPU`:

```
 master-replica-0 | Processes:                                                       GPU Memory | I  master-replica-0
 master-replica-0 |  GPU       PID   Type   Process name                             Usage      | I  master-replica-0
 master-replica-0 |=============================================================================| I  master-replica-0
 master-replica-0 |  No running processes found                                                 | I  master-replica-0
```

Hyperparameter tuning

In [4]:
%%writefile hyperparam.yaml
trainingInput:
  scaleTier: BASIC_GPU
  hyperparameters:
    maxTrials: 40
    maxParallelTrials: 5
    enableTrialEarlyStopping: False
    goal: MAXIMIZE    
    hyperparameterMetricTag: reward
    params:
    - parameterName: update_target
      type: INTEGER
      minValue: 500
      maxValue: 5000
      scaleType: UNIT_LOG_SCALE
    - parameterName: init_eta
      type: DOUBLE
      minValue: 0.8
      maxValue: 0.95
      scaleType: UNIT_LOG_SCALE
    - parameterName: learning_rate
      type: DOUBLE
      minValue: 0.00001
      maxValue: 0.001
      scaleType: UNIT_LOG_SCALE
    - parameterName: batch_size
      type: DISCRETE
      discreteValues:
      - 4
      - 16
      - 32
      - 64
      - 128
      - 256
      - 512

Overwriting hyperparam.yaml


In [5]:
%%bash
JOBNAME=rl_breakout_hp_$(date -u +%y%m%d_%H%M%S)
REGION='us-central1'
BUCKET='dqn-breakout'

gcloud ml-engine jobs submit training $JOBNAME \
        --package-path=$PWD/rl_on_gcp/trainer \
        --module-name=trainer.trainer \
        --region=$REGION \
        --staging-bucket=gs://$BUCKET \
        --config=hyperparam.yaml \
        --runtime-version=1.10 \
        --\
        --steps=100000\
        --start_train=10000\
        --buffer_size=10000\
        --model_dir='gs://dqn-breakout/models/hp/'

jobId: rl_breakout_hp_181128_175711
state: QUEUED


Job [rl_breakout_hp_181128_175711] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe rl_breakout_hp_181128_175711

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs rl_breakout_hp_181128_175711


### Launch tensorboard
Using `tensoboard --logdir='gs://dqn-breakout/models/'` or within datalab:

In [0]:
from google.datalab.ml import TensorBoard as tb

tb.start('gs://crawles-sandbox/rl_on_gcp/hp/4/')

In [1]:
!gsutil ls  gs://crawles-sandbox/rl_on_gcp/hp/4/

/bin/sh: gsutil: command not found
